In [1]:
!git clone https://github.com/nkt780426/3d_face_recognition_magface.git

Cloning into '3d_face_recognition_magface'...
remote: Enumerating objects: 601, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 601 (delta 38), reused 56 (delta 18), pack-reused 516 (from 1)
Receiving objects: 100% (601/601), 98.30 MiB | 40.12 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [2]:
!mv 3d_face_recognition_magface/going_modular .

In [3]:
import warnings
warnings.filterwarnings("ignore")
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A
import os
import torch

from going_modular.dataloader.multitask import create_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.loss.MultiTaskLoss import MultiTaskLoss
from going_modular.train_eval.train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    'type': 'normalmap',
    
    # Thư mục
    'dataset_dir': '/kaggle/input/hoangvn-3dmultitask',
    'checkpoint_dir': './checkpoint/multi/',
    
    # Cấu hình train
    'device': device,
    'epochs': 119,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'base_lr': 1e-4,
    
    # Cấu hình network
    'backbone': 'miresnet18',
    'embedding_size': 512,
    'num_classes': None,
    'loss_gender_weight': 30,
    'loss_da_gender_weight': 30,
    'loss_emotion_weight': 10,
    'loss_da_emotion_weight': 10,
    'loss_pose_weight': 30,
    'loss_da_pose_weight': 30,
    'loss_spectacles_weight': 5,
    'loss_da_spectacles_weight': 5,
    'loss_facial_hair_weight': 5,
    'loss_da_facial_hair_weight': 5,
}

CONFIGURATION['num_classes'] = len(os.listdir('/kaggle/input/hoangvn-3dmultitask/Albedo/train'))

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})


test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})

train_dataloader, test_dataloader, train_weight_class = create_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)
model = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])

criterion = MultiTaskLoss(os.path.join(CONFIGURATION['dataset_dir'], 'train_set.csv'), CONFIGURATION)
optimizer = Adam(model.parameters(), lr=CONFIGURATION['base_lr'])
# Khởi tạo scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models')

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'euclidean_auc'],
    patience=1000,
    mode='max',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=0
)      
checkpoint_path = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

In [4]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dataloader, 
    test_dataloader=test_dataloader, 
    criterion=criterion,
    optimizer=optimizer,    
    scheduler=scheduler, 
    early_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

Epoch 1:
╒═════════════════════╤════════════╤══════════════════════╕
│ Metric              │      Train │ Test                 │
╞═════════════════════╪════════════╪══════════════════════╡
│ loss                │ 66.1923    │ -                    │
├─────────────────────┼────────────┼──────────────────────┤
│ loss_id             │ 22.2507    │ -                    │
├─────────────────────┼────────────┼──────────────────────┤
│ loss_gender         │  0.0589373 │ 0.05908306874334812  │
├─────────────────────┼────────────┼──────────────────────┤
│ loss_da_gender      │  0.945521  │ 0.640736470464617    │
├─────────────────────┼────────────┼──────────────────────┤
│ loss_emotion        │  0.180473  │ 0.18095634132623672  │
├─────────────────────┼────────────┼──────────────────────┤
│ loss_da_emotion     │  0.434121  │ 0.5385885760188103   │
├─────────────────────┼────────────┼──────────────────────┤
│ loss_pose           │  0.0216272 │ 0.020633961772546172 │
├─────────────────────┼────────

In [5]:
!zip -r output.zip checkpoint/multi/

  adding: checkpoint/multi/ (stored 0%)
  adding: checkpoint/multi/normalmap/ (stored 0%)
  adding: checkpoint/multi/normalmap/models/ (stored 0%)
  adding: checkpoint/multi/normalmap/models/checkpoint.pth (deflated 19%)
  adding: checkpoint/multi/normalmap/models/best_cosine_auc_115.pth (deflated 7%)
  adding: checkpoint/multi/normalmap/models/best_euclidean_auc_86.pth (deflated 7%)
  adding: checkpoint/multi/normalmap/logs/ (stored 0%)
  adding: checkpoint/multi/normalmap/logs/AUC_euclidean_test/ (stored 0%)
  adding: checkpoint/multi/normalmap/logs/AUC_euclidean_test/events.out.tfevents.1736971006.cee9790d3f9a.18.24 (deflated 65%)
  adding: checkpoint/multi/normalmap/logs/events.out.tfevents.1736970944.cee9790d3f9a.18.0 (deflated 63%)
  adding: checkpoint/multi/normalmap/logs/Loss_spectacles_test/ (stored 0%)
  adding: checkpoint/multi/normalmap/logs/Loss_spectacles_test/events.out.tfevents.1736971006.cee9790d3f9a.18.6 (deflated 65%)
  adding: checkpoint/multi/normalmap/logs/Loss_em